In [38]:
import requests
import time

In [39]:
# Define base URL for API
BASE_URL = "http://localhost:8080/mktdata/session"

In [32]:
def decodeResponse(response : any) -> str:
    try:
        # Try to parse JSON, if it fails, handle the string response
        resp = response.json()
    except ValueError:  # If JSON decoding fails
        # Handle the case where response is a simple string
        resp = response.text
    return resp

def getSessionId(response):
    return decodeResponse(response)

In [40]:
# Step 1: Create a new session
def createSession() -> str:
    response = requests.post(BASE_URL)
    if response.status_code == 200:
        session_id = getSessionId(response)
        print(f"Session created: {session_id}")
    else:
        print(f"Failed to create session: {response.text}")
        session_id = None
    return session_id

In [41]:
# Step 2: subscribe to session
def subscribeSession(session_id : str) -> None:
    if session_id:
        start_url = f"{BASE_URL}/subscribe/{session_id}"
        response = requests.get(start_url, stream=True)
        print(f"Subscribe session response: {response.status_code}")

        # Process and display streaming data
        try:
            for chunk in response.iter_lines(decode_unicode=True):
                if chunk:
                    clear_output(wait=True)  # Clear previous output in notebook
                    display(f"Streaming data: {chunk}")  # Display the current chunk
                    time.sleep(0.1)  # Simulate delay between chunks
        except KeyboardInterrupt:
            print("Streaming stopped manually.")

In [42]:
# Step 3: Start the replay session
def startSession(session_id : str) -> None:
    if session_id:
        start_url = f"{BASE_URL}/start/{session_id}"
        response = requests.put(start_url)
        print(f"Start session response: {response.text}")

In [43]:
# Step 4: Rewind the session
def rewindSession(session_id : str) -> None:
    if session_id:
        rewind_url = f"{BASE_URL}/rewind/{session_id}"
        response = requests.put(rewind_url)
        print(f"Rewind session response: {response.text}")

In [44]:
# Step 4: Set replay speed to 2.0
def setSpeed(session_id : str, speed : float) -> None:
    if session_id:
        speed_url = f"{BASE_URL}/speed/{session_id}/{speed}"
        response = requests.put(speed_url)
        print(f"Set replay speed response: {response.text}")


In [52]:
sessions = []

# Create sessions
for sesssion_count in range(1, 100):
    sessionId = createSession()
    sessions.append(sessionId)


Session created: e07fd2e8-6216-430a-a4fb-a590c4f7d80d
Session created: aa3d61c9-7f63-43b9-8bce-15c51bb885c4
Session created: 2f687999-3797-43a2-b5f3-4019edddc43c
Session created: 7cd77c3c-07c2-4779-90e5-c2fe78879013
Session created: 1a8483cc-eb92-4e8c-af4c-f14fe58666e9
Session created: be8281d3-65ce-4194-acc8-496b24e23f45
Session created: 1fd4ad19-78a9-46e8-9e92-5ed9c811851e
Session created: b231cc77-2cc2-4b1a-9644-601e3bfdef97
Session created: 67106ef8-92c4-4b27-b4b3-f1d6f23f88b5
Session created: 3a8e00a3-112f-43f5-8396-ace9c3258c2e
Session created: 28dbfb06-6f33-4579-be2a-6a230c3acf30
Session created: cfddc726-3a9d-4376-ae4c-311bfb062c9b
Session created: 57aff6fc-e674-442c-8248-64cffc59692a
Session created: 83fe9dbb-ec3c-42a1-b309-9b598090204b
Session created: cb39cd8c-00ee-4241-9047-67a8dff5d6f9
Session created: 9b4d385c-d6ae-4fbb-b8ad-5fa5c89c95d2
Session created: db9e159e-45a6-4021-ab11-9c948f4787ef
Session created: f6cab758-83fb-43a2-b5d5-6ffb09159275
Session created: 44b62409-e2

In [53]:
# Set speed
for session in sessions:
    setSpeed(session, 1000.0)

Set replay speed response: Replay speed set to 1000.0 for session e07fd2e8-6216-430a-a4fb-a590c4f7d80d
Set replay speed response: Replay speed set to 1000.0 for session aa3d61c9-7f63-43b9-8bce-15c51bb885c4
Set replay speed response: Replay speed set to 1000.0 for session 2f687999-3797-43a2-b5f3-4019edddc43c
Set replay speed response: Replay speed set to 1000.0 for session 7cd77c3c-07c2-4779-90e5-c2fe78879013
Set replay speed response: Replay speed set to 1000.0 for session 1a8483cc-eb92-4e8c-af4c-f14fe58666e9
Set replay speed response: Replay speed set to 1000.0 for session be8281d3-65ce-4194-acc8-496b24e23f45
Set replay speed response: Replay speed set to 1000.0 for session 1fd4ad19-78a9-46e8-9e92-5ed9c811851e
Set replay speed response: Replay speed set to 1000.0 for session b231cc77-2cc2-4b1a-9644-601e3bfdef97
Set replay speed response: Replay speed set to 1000.0 for session 67106ef8-92c4-4b27-b4b3-f1d6f23f88b5
Set replay speed response: Replay speed set to 1000.0 for session 3a8e00a

In [54]:
# Start sessions
for session in sessions:
    startSession(session)

Start session response: Replay started for session e07fd2e8-6216-430a-a4fb-a590c4f7d80d
Start session response: Replay started for session aa3d61c9-7f63-43b9-8bce-15c51bb885c4
Start session response: Replay started for session 2f687999-3797-43a2-b5f3-4019edddc43c
Start session response: Replay started for session 7cd77c3c-07c2-4779-90e5-c2fe78879013
Start session response: Replay started for session 1a8483cc-eb92-4e8c-af4c-f14fe58666e9
Start session response: Replay started for session be8281d3-65ce-4194-acc8-496b24e23f45
Start session response: Replay started for session 1fd4ad19-78a9-46e8-9e92-5ed9c811851e
Start session response: Replay started for session b231cc77-2cc2-4b1a-9644-601e3bfdef97
Start session response: Replay started for session 67106ef8-92c4-4b27-b4b3-f1d6f23f88b5
Start session response: Replay started for session 3a8e00a3-112f-43f5-8396-ace9c3258c2e
Start session response: Replay started for session 28dbfb06-6f33-4579-be2a-6a230c3acf30
Start session response: Replay s

In [ ]:
# # Subscribe to sessions
# for session in sessions:
#     subscribeSession(session)